#Motorhang felismerő

Célja:
Előre megtanított hibák detektálása a motor hangjából

Adatok: Előre felvett hangminták.
1. Alapjárat ~60mp (2.5M adatpont)
2. Első gyertya kihagy ~30mp (1.25M adatpont)
3. Második gyertya kihagy ~30mp (1.25M adatpont)

Normál alapjárati hangminta:
<img src='https://raw.githubusercontent.com/badinszky97/Kurt_AI_tech/main/Pics/alapjarat.png' />

Hangminta ha az első gyetya kihagy:
<img src='https://raw.githubusercontent.com/badinszky97/Kurt_AI_tech/main/Pics/1lehuzva.png' />

Első ránézésre, nem látni szabad szemmel a különbséget, de van némi priori tudásunk a hangokról:
1. Motorfordulat: ~850rpm
2. Mikrofon mintavételezése: 44100Hz

Ezekből kiszámolható, hogy körülbelül 1500 adatpontban benne kell lennie mind a 4 henger egyszeri gyujtásának.

1500 -> Ablakméret

## Adatfeldolgozás

### Ablakok

Egy ~60mp-es hangfájlt ablakméretnyi elemekre bontunk. Ehhez két elvet követtem:

1. Az ablakok szorosan követik egymást (kevesebb minta, de nincs ismétlődés)
2. **Az ablakok átfedésben vannak (több minta, ismétlődés van benne, de a a robbanások sokkal változatosabban fordulnak elő ablakokon belül)**


### Zajszűrés

2 (+1) féle módszer tűnt kivitelezhetőnek:

1. Egy python library-ból történő zajszűrő algoritmus
2. Az adatpontokok átagolása fix szélességben. (10 pontnyi)
3. **Néha sok sok munka van benne, hogy zajt vigyünk egy rendszerbe. Örüljünk, hogy benne van :)**

In [ ]:
def WavToOverlappedArray(filename="", window_size=1500, overlap = 500):
    fs, wave = scipy.io.wavfile.read(filename) 
    
    kimenet = []
    i = 0
    while i+1*window_size-overlap < len(wave):
        kimenet.append(   wave[(i+1*window_size-overlap)  :  (i+2*window_size-overlap)].tolist()   )
        i = i+overlap    
    return kimenet



class DataKeeper:
    def __init__(self):
        self.RawDatas = []
        self.RawLabel = []
    def AddNewType(self, data, label):
        self.RawDatas.append(data)
        self.RawLabel.append(label)
    
    @property
    def KeptTypeCount(self):
        return len(self.RawDatas)
    
    def GetData(self, index):
        return self.RawDatas[index]
    
    def GenerateTrainTestValid(self, alpha=0.8):
        #if aplha is 0.8 -> train is 80%, valid is 10%, test is 10%
        SumRawDatas = []
        #létrehozzuk a teljes adatállományt egy tömbben
        for x in range(0, len(self.RawDatas)):
            for element in self.RawDatas[x]:
                SumRawDatas.append([element,self.RawLabel[x]])
        #összekeverjük
        random.shuffle(SumRawDatas)
        #szétválogatjuk 3 részre
        train = SumRawDatas[:int(len(SumRawDatas)*alpha)]
        test_valid = SumRawDatas[int(len(SumRawDatas)*alpha):]
        test = test_valid[:int(len(test_valid)*0.5)]
        valid = test_valid[int(len(test_valid)*0.5):]
        
        #x_train, y_train létrehozása
        self.x_train = []
        self.y_train = []
        for x in train:
            self.x_train.append(x[0])
            self.y_train.append(x[1])
            
        #x_test, y_test létrehozása
        self.x_test = []
        self.y_test = []
        for x in test:
            self.x_test.append(x[0])
            self.y_test.append(x[1])
            
        #x_valid, y_valid létrehozása
        self.x_valid = []
        self.y_valid = []
        for x in train:
            self.x_valid.append(x[0])
            self.y_valid.append(x[1])
            
        #numpy tömbbe konvertálás
        self.x_train = np.array(self.x_train)
        self.y_train = np.array(self.y_train)
        
        self.x_test = np.array(self.x_test)
        self.y_test = np.array(self.y_test)
        
        self.x_valid = np.array(self.x_valid)
        self.y_valid = np.array(self.y_valid) 
        print("Kész")

Ezt a kódot használva a classok felvétele ilyen egyszerű:

In [ ]:
Adatgyujto.AddNewType(WavToOverlappedArray("sounds/1lehuzva_mono.wav", ablakmeret, 500), 0)
Adatgyujto.AddNewType(WavToOverlappedArray("sounds/2lehuzva_mono.wav", ablakmeret, 500), 1)
Adatgyujto.AddNewType(WavToOverlappedArray("sounds/alapjarat_mono.wav", ablakmeret, 500),2)

#AI Modellek

5 féle modelt készítettem és a terv szerint a gyakorlat megmutatja melyik elég gyors és pontos a használathoz.

1. RandomForest
2. Egyszerű neurális háló
3. Bonyolult neurális háló
4. LSTM
5. 1D Konvolúciós háló

#RandomForest

A Klasszikus módszerek Szent Grálja. Működését figyelembe véve nem biztos, hogy alkalmas lesz a feladat megoldására, de hátha!



In [ ]:
ablakmeret = 1500

Adatgyujto = DataKeeper()

Adatgyujto.AddNewType(  WavToOverlappedArray("sounds/1lehuzva_mono.wav", ablakmeret, 500)  , 0)
Adatgyujto.AddNewType(  WavToOverlappedArray("sounds/2lehuzva_mono.wav", ablakmeret, 500)  , 1)
Adatgyujto.AddNewType(  WavToOverlappedArray("sounds/alapjarat_mono.wav", ablakmeret, 500) , 2)

Adatgyujto.GenerateTrainTestValid()

model = RandomForestClassifier(n_estimators=100)
model.fit(X=Adatgyujto.x_train, y=Adatgyujto.y_train)

A tanítás végén a model 73%-ot teljesített a valid adatokon. Nagyon ígéretes!

#Egyszerű neurális háló

A kérdés adott: Képes-e eltolt mintákat felismerni egy tömbben?

In [ ]:
hidden_layer_1_size = 50
hidden_layer_2_size = 30


tf.compat.v1.reset_default_graph()
clear_session            


# Model
#######

x = Input(shape=(ablakmeret,))

# Hidden layers

d0_layer = Dropout(.3)(x)  #érdekes, de működik
hidden_layer_1 = Dense(units=hidden_layer_1_size, activation="relu", kernel_regularizer="l2")(d0_layer)
d1_layer = Dropout(.2)(hidden_layer_1)
hidden_layer_2 = Dense(units=hidden_layer_2_size, activation="relu", kernel_regularizer="l2")(d1_layer)
d2_layer = Dropout(.1)(hidden_layer_2)
                           
# Softmax 
predictions = Dense(Adatgyujto.KeptTypeCount, activation="softmax")(d2_layer)

# Full model
model = Model(inputs=x, outputs=predictions)
model.summary()

<img src='https://raw.githubusercontent.com/badinszky97/Kurt_AI_tech/main/Overlap/KerasModels/SimpleDense_Acc.png' />


Tanítás után a model 71%-ot teljesített a valid adatokon.

#Bonyolult neurális háló

Az kód az előzővel azonos, de a két réteg mérete: 150,50

<img src="https://raw.githubusercontent.com/badinszky97/Kurt_AI_tech/main/Overlap/KerasModels/DeepDense_Acc.png" />

#LSTM

Az első olyan technika amiről tudjuk, hogy elviekben képes lehet megoldani a feladatot.

In [ ]:
tf.compat.v1.reset_default_graph()
clear_session            
lstm_size = 20

# Model
#######

input_shape=(ablakmeret, 1)

x = Input(shape=input_shape)

# Hidden layers

#d0_layer = Dropout(.3)(x)
lstmlayer = LSTM(lstm_size, return_sequences=True)(x)
lstmlayer2 = LSTM(lstm_size, return_sequences=False)(lstmlayer)


dense1 = Dense(units=50, activation="relu")(lstmlayer2)
#dropout3 = Dropout(rate=0.3)(dense1)
dense2 = Dense(units=30, activation="relu")(dense1)
#dropout4 = Dropout(rate=0.3)(dense2)
predictions = Dense(units = 3, activation='softmax')(dense2)

model = Model(inputs=x, outputs=predictions)

model.summary()

Sajnos a tanulási folyamat így néz ki:
<img src="https://raw.githubusercontent.com/badinszky97/Kurt_AI_tech/main/Overlap/KerasModels/lstm.png">

Úgy tűnik a tanulás során beragadt egy állapotba....

#1D Konvolúciós háló

A legígéretesebb technika erre a célra. A képekben alkalmazott mintakeresése itt is hasznos lehet.

In [ ]:
tf.compat.v1.reset_default_graph()
clear_session            


# Model
#######

input_shape=(ablakmeret, 1)

x = Input(shape=input_shape)

# Hidden layers

#d0_layer = Dropout(.3)(x)
conv1 = Conv1D(filters=40,
                kernel_size=5,
                activation="relu", input_shape=input_shape[1:])(x)

pool1 = MaxPool1D(pool_size=64)(conv1)

#dropout1 = Dropout(rate=0.3)(pool1)
conv2 = Conv1D(filters=10,
               kernel_size=3,
               activation="relu")(pool1)

pool2 = MaxPool1D(pool_size=20, strides=1)(conv2)
fllayer = Flatten()(pool2)

dense1 = Dense(units=100, activation="relu")(fllayer)
#dropout3 = Dropout(rate=0.1)(dense1)
dense2 = Dense(units=20, activation="relu")(dense1)
#dropout4 = Dropout(rate=0.1)(dense2)
predictions = Dense(units = 3, activation='softmax')(dense2)

model = Model(inputs=x, outputs=predictions)

model.summary()

#71,4%:  20,100  10,10  2,2    100    0.3   50   0.3
#71,4%:  40,5    10,3   2,2    100    0,7   20   0.5

A tanítás így alakult: <img src="https://raw.githubusercontent.com/badinszky97/Kurt_AI_tech/main/Overlap/KerasModels/1D_conv_long.png">

#A végeredménye: 98%

#Production környezet

A működés: Egy mikrofon felvesz adott mennyiségű adatpontot (15000 pont) és párhuzamosan futtatja az 5 modellen. Ekkor látszik, mennyire értenek egyet.
Mivel a modellek alapból párhuzamosítottak egymaguk is, így inkább a modeleket egymás után sorban hagytam érvényesülni.

A gyakorlatban kiderült, hogy ez nagyon erőforrásigényes :) Így végül csak 4 model lett futtava:

In [ ]:
Model1DConvNet = keras.models.load_model('KerasModels/1DConvNet.h5')
Model1DConvNet.summary()

ModelDeepDenseNet = keras.models.load_model('KerasModels/DeepDenseNet.h5')
ModelDeepDenseNet.summary()

ModelLSTM = keras.models.load_model('KerasModels/LSTM.h5')
ModelLSTM.summary()

ModelSimpleDenseNet = keras.models.load_model('KerasModels/SimpleDenseNet.h5')
ModelSimpleDenseNet.summary()

clear = lambda: os.system('clear')

def Encode(mode, input):
	labels = ["1 Lehuzva", "2 Lehuzva", "Alapjarat"]
	print(mode + " " + labels[argmax(input)])
	
while(True):
#	clear()
	record = sd.rec(int((1/2.94) * 44100), samplerate=44100, channels=1)

	
	Encode("SimpleDense ",  ModelSimpleDenseNet.predict(record[:1500].reshape(1,1500))   )
	Encode("DeepDense   ",  ModelDeepDenseNet.predict(record[:1500].reshape(1,1500))   )
	Encode("LSTM        ",  ModelLSTM.predict(record.reshape(1,15000))   )
	Encode("1DConvNet   ",  Model1DConvNet.predict(record.reshape(1,15000))   )
	K.clear_session()
